# Advent of Code 2021 - Day 24

In [151]:
data class ALU(val w: Int, val x: Int, val y: Int, val z: Int) {
    enum class Register { W, X, Y, Z }
}

sealed interface Instruction { val lhs: Value.Register }
data class Input(override val lhs: Value.Register) : Instruction

sealed interface Operation : Instruction { val rhs: Value }
data class Add(override val lhs: Value.Register, override val rhs: Value) : Operation
data class Multiply(override val lhs: Value.Register, override val rhs: Value) : Operation
data class Divide(override val lhs: Value.Register, override val rhs: Value) : Operation
data class Modulo(override val lhs: Value.Register, override val rhs: Value) : Operation
data class Equal(override val lhs: Value.Register, override val rhs: Value) : Operation

sealed interface Value {
    @JvmInline value class Register(val value: ALU.Register) : Value
    @JvmInline value class Integer(val value: Int) : Value
}

In [152]:
import java.io.File

fun String.toValue(): Value = when (this) {
    "w" -> Value.Register(ALU.Register.W)
    "x" -> Value.Register(ALU.Register.X)
    "y" -> Value.Register(ALU.Register.Y)
    "z" -> Value.Register(ALU.Register.Z)
    else -> Value.Integer(toInt())
}
fun String.toRegister(): Value.Register = (toValue() as? Value.Register) ?: TODO()

val regex = Regex("""([a-z]{3}) ([wxyz])\s*([wxyz]|-?\d+)?""")

val instructions: List<Instruction> = File("Day24.input.txt")
    .bufferedReader()
    .readLines()
    .map { regex.find(it)!!.destructured }
    .map { (instruction, lhs, rhs) ->
        if (instruction == "inp") Input(lhs.toRegister())
        else if (instruction == "add") Add(lhs.toRegister(), rhs.toValue())
        else if (instruction == "mul") Multiply(lhs.toRegister(), rhs.toValue())
        else if (instruction == "div") Divide(lhs.toRegister(), rhs.toValue())
        else if (instruction == "mod") Modulo(lhs.toRegister(), rhs.toValue())
        else if (instruction == "eql") Equal(lhs.toRegister(), rhs.toValue())
        else TODO()
    }

## Part 1

Decode the input and determine the largest model number.

In [153]:
operator fun ALU.get(register: ALU.Register): Int = when (register) {
    ALU.Register.W -> w
    ALU.Register.X -> x
    ALU.Register.Y -> y
    ALU.Register.Z -> z
}

operator fun ALU.get(value: Value): Int = when (value) {
    is Value.Register -> get(value.value)
    is Value.Integer -> value.value
}

fun ALU.set(register: ALU.Register, value: Int): ALU = when (register) {
    ALU.Register.W -> copy(w = value)
    ALU.Register.X -> copy(x = value)
    ALU.Register.Y -> copy(y = value)
    ALU.Register.Z -> copy(z = value)
}

fun ALU.set(register: Value.Register, value: Int): ALU = set(register.value, value)

In [154]:
fun ALU.process(instruction: Instruction): ALU = when (instruction) {
    is Add -> set(instruction.lhs, get(instruction.lhs) + get(instruction.rhs))
    is Multiply -> set(instruction.lhs, get(instruction.lhs) * get(instruction.rhs))
    is Divide -> set(instruction.lhs, get(instruction.lhs) / get(instruction.rhs))
    is Modulo -> set(instruction.lhs, get(instruction.lhs) % get(instruction.rhs))
    is Equal -> set(instruction.lhs, if (get(instruction.lhs) == get(instruction.rhs)) 1 else 0)
    is Input -> this
    else -> TODO()
}

fun ALU.process(instructions: List<Instruction>): ALU = instructions.fold(this) { prev, it -> prev.process(it) }

In [155]:
fun <T> List<T>.partitionStart(condition: (T) -> Boolean): List<List<T>> =
    buildList<MutableList<T>> {
        var index = -1
        this@partitionStart.forEach { element ->
            if (condition(element)) add(mutableListOf(element)).also { index++ }
            else getOrNull(index)?.let { it.add(element) }
        }
    }

fun List<List<Instruction>>.modelNumber(smallest: Boolean = false): String {
    val modelNumber = mutableListOf(0,0, 0,0,0, 0,0,0, 0,0,0, 0,0,0)
    val stack = mutableListOf<Pair<Int, Int>>()

    (0..13).forEach {
        val addX = ((this[it][5] as? Add)?.rhs as? Value.Integer)?.value ?: TODO()
        if (addX > 0) {
            val addY = ((this[it][15] as? Add)?.rhs as? Value.Integer)?.value ?: TODO()
            stack.add(0, addY to it)
        } else {
            val (addY, digit) = stack.removeFirst()
            val current = if (smallest) generateSequence(1) { it + 1 }.first { it + addY + addX >= 1 }
                else generateSequence(9) { it - 1 }.first { it + addY + addX <= 9 }
            modelNumber[digit] = current
            modelNumber[it] = current + addY + addX
        }
    }
    return modelNumber.joinToString("")
}

instructions.partitionStart { it is Input }.modelNumber()

92969593497992

### Notes

After running the program for 10 minutes, figured I needed to do the "You'll need to figure out what MONAD does some other way." part of the problem.

Some initial things I noticed is that each subdivided instruction has a similar pattern:
- 18 instructions long, starts with `w` input.
- sets the value of `x` to `z`. -- `x` is dependent on calculation from the previous `z` (`0` if valid)
- mod value of `x` by `26` -- this might represent a character, `0` if previous was valid
- divide `z` -- `0` if previous was valid
  - [1,2,3,4,7,9,11] by `1` -- `z` is unchanged
  - [5,6,8,10,12,13,14] by `26` -- `z`'s remainder is `x`
- add `x` -- the rhs value or `z`'s remainder added by the following
  - [1,2,4,7] by `11`
  - [3,11] by `14`
  - [5,14] by `-8`
  - [6,12] by `-5`
  - [8] by `-13`
  - [9] by `12`
  - [10] by `-1`
  - [13] by `-4`
- check if `x` DOES NOT equal to `w` -- `1` (does not) or `0` (does)
- set `y` to `25` -- introduce y
- multiply `y` by `x` -- `0` if `x == w` else `25`
- add `y` by `1` -- `1` if `x == w` else `26`
- multiply `z` by `y` -- `0` if previous was valid, unchanged if `x == w`
- set `y` to `w`
- add `y` -- add the input by the following value
  - [1,3,12] by `1`
  - [2,4,8] by `11`
  - [5] by `2`
  - [6] by `9`
  - [7,11] by `7`
  - [9,14] by `6`
  - [10] by `15`
  - [13] by `8`
- multiply `y` by `x` -- `0` if `x == w`, unchanged if `x != w`
- add `z` by `y` -- unchanged if `x == w`

After walking through the program, it looks like the program is a stack and I can code out just the variables that change 🤔.

---

I felt pretty betrayed by this problem since its "trick" was reading and understanding the underlying input. I spent a ton of time walking through it by hand before realizing what it was. Basically all the code for the ALU was useless and only a small portion of the input was "required." This was a scenario in which the problem statement was actually the input. Ultimately, breaking down the input enables you to write the "code" version of the input.

## Part 2

Find the smallest model number.

In [156]:
instructions.partitionStart { it is Input }.modelNumber(smallest = true)

81514171161381

### Notes

Ended up making a small change to the function written in part 1 to find this.